In [1]:
!pip freeze > requirements.txt

### Импорт библиотек

In [2]:
import sys

from nltk.tokenize import word_tokenize
import numpy as np
from rank_bm25 import BM25Okapi
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import torch
import transformers
from transformers import BertTokenizer, BertForSequenceClassification

print('python:'.ljust(16), sys.version.split('\n')[0])
print('scikit-learn:'.ljust(16), sklearn.__version__)
print('pytorch:'.ljust(16), torch.__version__)
print('transformers:'.ljust(16), transformers.__version__)

python:          3.10.9 (v3.10.9:1dd9be6584, Dec  6 2022, 14:37:36) [Clang 13.0.0 (clang-1300.0.29.30)]
scikit-learn:    1.3.1
pytorch:         2.0.1
transformers:    4.33.2


### Создание корпуса и запроса

In [3]:
corpus = [
    "Мороз и солнце, день чудесный.",
    "Я помню чудное мгновенье:",
    "Передо мной явилась ты,",
    "Как мимолетное виденье,",
    "Как гений чистой красоты.",
    "что-то нерелевантное",
]

query = "Мороз и солнце, день чудесный. Я помню чудное мгновенье."

### Задание 1

Создайте экземпляр класса BM25Okapi и отранжируйте с его помощью предложения из списка corpus по отношению к запросу query.

Предложения из corpus отсортируйте по полученным bm_ranking_scores и выведите на печать в формате: <предложение> - <значение score>

In [4]:
def print_result(corpus, scores):
    # Сортируем корпус по значениям 'scores'
    pairs = [(doc, score) for doc, score in zip(corpus, scores)]
    pairs = sorted(pairs, key=lambda pair: pair[1], reverse=True)
    
    # Выводим на печать
    for i, pair in enumerate(pairs, start=1):
        print(f'{i}. {pair[0]}'.ljust(35), 'Score:', pair[1])

In [5]:
# Токенизируем корпус и запрос
tokenized_corpus = [word_tokenize(doc, language='russian') for doc in corpus]
tokenized_query = word_tokenize(query, language='russian')

# Рассчитываем значения BM25
bm25 = BM25Okapi(tokenized_corpus)
bm25_scores = bm25.get_scores(tokenized_query).tolist()

# Выводим результат на печать
print('Ранжирование по BM25:')
print_result(corpus, bm25_scores)

Ранжирование по BM25:
1. Мороз и солнце, день чудесный.   Score: 6.26284755139228
2. Я помню чудное мгновенье:        Score: 5.0352835371138145
3. Как гений чистой красоты.        Score: 1.1389637797411305
4. Передо мной явилась ты,          Score: 0.0
5. Как мимолетное виденье,          Score: 0.0
6. что-то нерелевантное             Score: 0.0


### Задание 2

Создайте экземпляр класса BertForSequenceClassification (с весами DeepPavlov/rubert-base-cased-conversational) и отранжируйте 
с его помощью предложения из списка corpus по отношению к запросу query на основании cosine_similarity.

Предложения из corpus отсортируйте по полученным bert_ranking_scores и выведите на печать в формате: <предложение> - <значение score>

#### Решение на основе CLS токена

In [6]:
# Загружаем модель из репозитория Hugging Face
model_checkpoint = 'DeepPavlov/rubert-base-cased-conversational'
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
model = BertForSequenceClassification.from_pretrained(model_checkpoint)

# Токенизируем корпус и запрос
encoded_corpus = tokenizer(corpus, padding=True, return_tensors='pt')
encoded_query = tokenizer(query, return_tensors='pt')

# Рассчитываем скрытые состояния модели для корпуса и запроса
with torch.no_grad():
    corpus_hidden_states = model(**encoded_corpus, output_hidden_states=True)
    query_hidden_states = model(**encoded_query, output_hidden_states=True)

# Берем скрытое состояние последнего слоя
corpus_last_hidden_state = corpus_hidden_states.hidden_states[-1]
query_last_hidden_state = query_hidden_states.hidden_states[-1]

# Берем скрытое состояние для CLS токена
corpus_embeds = corpus_last_hidden_state[:, 0, :]
query_embeds = query_last_hidden_state[:, 0, :]

# Рассчитываем косинусную близость
cosine_scores = cosine_similarity(corpus_embeds, query_embeds).squeeze().tolist()

# Выводим результат на печать
print()
print('Ранжирование по косинусной близости (на основе CLS токена):')
print_result(corpus, cosine_scores)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Ранжирование по косинусной близости (на основе CLS токена):
1. Мороз и солнце, день чудесный.   Score: 0.9423289895057678
2. Как гений чистой красоты.        Score: 0.8802600502967834
3. Я помню чудное мгновенье:        Score: 0.8675134778022766
4. Как мимолетное виденье,          Score: 0.33058974146842957
5. что-то нерелевантное             Score: 0.3149392008781433
6. Передо мной явилась ты,          Score: 0.1957339346408844


#### Решение на основе mean pooling

In [7]:
def mean_pooling(last_hidden_state, attention_mask):
    expanded_attention_mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    embedding_sum = torch.sum(last_hidden_state * expanded_attention_mask, 1)
    mask_sum = torch.clamp(expanded_attention_mask.sum(1), min=1e-9)
    return embedding_sum / mask_sum

In [8]:
# Рассчитываем mean pooling для корпуса и запроса
corpus_embeds = mean_pooling(corpus_last_hidden_state, encoded_corpus['attention_mask'])
query_embeds = mean_pooling(query_last_hidden_state, encoded_query['attention_mask'])

# Рассчитываем косинусную близость
cosine_scores = cosine_similarity(corpus_embeds, query_embeds).squeeze().tolist()

# Выводим результат на печать
print('Ранжирование по косинусной близости (на основе mean pooling):')
print_result(corpus, cosine_scores)

Ранжирование по косинусной близости (на основе mean pooling):
1. Мороз и солнце, день чудесный.   Score: 0.8117928504943848
2. Я помню чудное мгновенье:        Score: 0.7414984107017517
3. Как гений чистой красоты.        Score: 0.3663472831249237
4. Передо мной явилась ты,          Score: 0.357535183429718
5. Как мимолетное виденье,          Score: 0.32644036412239075
6. что-то нерелевантное             Score: 0.2974230945110321
